In [24]:
import tflearn
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn import preprocessing
import chess_game as cg

In [20]:
X = tflearn.layers.core.input_data(shape=(None, 71), name='inputs')
Y = tflearn.layers.core.input_data(shape=(None, 3), name='labels')

board_input, aux = tf.split(X, [64, 7], 1)
board_input_reshaped = tf.reshape(board_input, [-1, 8, 8, 1])

conv4 = tflearn.conv_2d(board_input_reshaped, 32, 4, activation='relu')


conv2 = tflearn.conv_2d(board_input_reshaped, 32, 2, activation='relu')
conv2_4 = tflearn.conv_2d(conv2, 64, 4, activation='relu')
# note: this is how to get separate branches of a larger NN and then have them converge
fc1 = tflearn.layers.core.fully_connected(tf.concat([conv4, conv2_4], 3), n_units=1000, activation='relu')

fc2 = tflearn.layers.core.fully_connected(tf.concat([fc1, aux], 1), n_units=2000, activation='relu')

net = tflearn.layers.core.fully_connected(fc2, n_units=3, activation='relu')

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=net, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.000005).minimize(loss)

init = tf.global_variables_initializer()
#net = tflearn.regression(net, optimizer='adam', metric=loss,
#                         loss='categorical_crossentropy')
#model = tflearn.DNN(net, tensorboard_verbose=0)


In [19]:
x,y = get_training_data(a)
model.fit(x, y, n_epoch=10)

IndexError: list index out of range

In [39]:
a = pd.read_csv('board_positions/board_positions_1.csv')
a = a[~a.isnull().any(axis=1)]
a = a[a['moves_in_game'] - a['move_num']<10]
le = preprocessing.LabelEncoder()
le.fit(['k', 'q', 'r', 'b', 'n', 'p', ' ', 'P', 'N', 'B', 'R', 'Q', 'K'])

bcols = a.columns[:64]

a[bcols] = le.transform(a[bcols])

def get_training_data(data):
    
    d = data.sample(100)
    train = d[d.columns[:71]]
    labels = np.zeros((100,3))
    labels[d['winner']==-1, 0] = 1
    labels[d['winner']==0, 1] = 1
    labels[d['winner']==1, 2] = 1
    
    return train, labels
    

In [42]:
sess = tf.Session()
sess.run(init)

batch_size = 128
for epoch in range(10):  # 2 epochs
    for i in range(200):
        batch_xs, batch_ys = get_training_data(a)
        sess.run(optimizer, feed_dict={X: batch_xs, Y: batch_ys})
        cost = sess.run(loss, feed_dict={X: batch_xs, Y: batch_ys})

        if i % 20 == 0:
            print("Epoch:", '%03d' % (epoch + 1), "Step:", '%03d' % i,
                  "Loss:", str(cost))

Epoch: 001 Step: 000 Loss: 1.0873935
Epoch: 001 Step: 020 Loss: 0.998387
Epoch: 001 Step: 040 Loss: 0.96666384
Epoch: 001 Step: 060 Loss: 0.9183046
Epoch: 001 Step: 080 Loss: 0.9286258
Epoch: 001 Step: 100 Loss: 0.9899873
Epoch: 001 Step: 120 Loss: 0.89291877
Epoch: 001 Step: 140 Loss: 0.845524
Epoch: 001 Step: 160 Loss: 0.9148894
Epoch: 001 Step: 180 Loss: 0.86016953
Epoch: 002 Step: 000 Loss: 0.9632863
Epoch: 002 Step: 020 Loss: 0.9083745
Epoch: 002 Step: 040 Loss: 0.8824754
Epoch: 002 Step: 060 Loss: 0.9044097
Epoch: 002 Step: 080 Loss: 0.8695275
Epoch: 002 Step: 100 Loss: 0.8448075
Epoch: 002 Step: 120 Loss: 0.8836602
Epoch: 002 Step: 140 Loss: 0.7595635
Epoch: 002 Step: 160 Loss: 0.8684771
Epoch: 002 Step: 180 Loss: 0.8664125
Epoch: 003 Step: 000 Loss: 0.8010799
Epoch: 003 Step: 020 Loss: 0.83733374
Epoch: 003 Step: 040 Loss: 0.7959285
Epoch: 003 Step: 060 Loss: 0.7898638
Epoch: 003 Step: 080 Loss: 0.83782786
Epoch: 003 Step: 100 Loss: 0.7828215
Epoch: 003 Step: 120 Loss: 0.815375

In [56]:
b = cg.game_board()
next_positions = b.find_all_next_board_positions('W')
next_positions_ar = np.array([ele.to_csv_format() for ele in next_positions])
next_positions_ar[:, :64] = le.transform(next_positions_ar[:, :64])


In [92]:
next_positions = b.find_all_next_board_positions('W')
next_positions_ar = np.array([ele.to_csv_format() for ele in next_positions])
next_positions_ar[:, :64] = le.transform(next_positions_ar[:, :64])

next_scores = sess.run(net, feed_dict={X:next_positions_ar})
b = next_positions[np.argmax(next_scores[:,2])]

In [93]:
print(b)

| r | n | b |   | k | b |   | r
| p | p | p | p |   | p |   | p
|   |   |   |   |   |   |   |  
|   |   |   |   | q |   |   |  
|   |   |   |   | P | p | P |  
|   |   |   |   |   |   |   |  
| P | P | P |   |   |   | B | P
| R | N |   | K |   |   | N | R


In [90]:
b.move('Qxe5')

In [91]:
print(b)

| r | n | b |   | k | b |   | r
| p | p | p | p |   | p |   | p
|   |   |   |   |   |   |   |  
|   |   |   |   | q |   |   |  
|   |   |   |   | P | p | P |  
|   |   |   |   |   |   |   |  
| P | P | P |   |   |   |   | P
| R | N |   | K |   | B | N | R


In [55]:
import importlib
importlib.reload(cg)

<module 'chess_game' from '/Users/pschale/pythonstuff/chess_ai_project/chess_game.py'>

In [4]:
a = pd.read_csv('board_positions/board_positions_1.txt')

In [11]:
a[a['moves_in_game'] - a['move_num']<10][a.columns[:71]]

,a1,a2,a3,a4,a5,a6,a7,a8,b1,b2,...,h6,h7,h8,white_tomove,w_castle_kingside,w_castle_queenside,b_castle_kingside,b_castle_queenside,w_material,b_material
50,,P,,,,,p,,R,,...,,p,,0,0,0,0,0,15,16
51,,P,,,,,p,,R,,...,,p,,1,0,0,0,0,12,16
52,,P,,,,,p,,,,...,,p,,0,0,0,0,0,12,16
53,,P,,,,,p,,,,...,,p,,1,0,0,0,0,12,16
54,,P,,,,,p,,,,...,,p,,0,0,0,0,0,12,16
55,,P,,,,,p,,,,...,,,,1,0,0,0,0,12,16
56,,P,,,,,p,,,,...,,,,0,0,0,0,0,12,16
57,,P,,,,,p,,,,...,,,,1,0,0,0,0,11,16
58,,P,,,,,p,,,,...,,,,0,0,0,0,0,11,15
59,,r,,,,,p,,,,...,,,,1,0,0,0,0,10,15


In [ ]:
tf.__version__